This notebook takes the raw Presidential Elections spreadsheet in elections.csv and converts it into a GoogleDataTable, output as the file presidential_data_google.json.  This involves:
1. Splitting the combined field \<candidateName> - \<party> into two fields, candidate and party
2. Converting years to integers and putting in the missing years (converting '2016', '', '' to 2016, 2016, 2016
3. Collecting the cells of a particular state and year into a structure, with the individual candidates as a list
4. Converting the votes into integers, and then, for each result, adding a percentage float
5. generating the individual records (state, year, candidate, party, votes, percentage) as a list
6. adding the description
7. creating the data table
8. Writing this out as a JSON file

Install the google visualization API and import it.

Please enter the room in the variable room:

In [100]:
room = 'testRoom1'

In [101]:
!pip install --upgrade gviz_api
import gviz_api
import json

Requirement already up-to-date: gviz_api in /opt/conda/lib/python3.7/site-packages (1.9.0)


Import the client, put in the room (by hand, this will change) and get a connection.  The room must be the same as for the dashboard.

In [102]:
import test_client
client = test_client.Client()
test_client.log = []
client.connect()

A utility to get the client to send the data to the dashboard

In [103]:
def send_data_to_dashboard(table_name, data_table_as_JSON, client, room):
    table_record = '{"name": "%s", "table": %s}' % (table_name, data_table_as_JSON)
    client.send(table_record, room)

Step 1: Read in the CSV

In [104]:
import csv
f = open('elections.csv', 'r')
election_reader = csv.reader(f)
rows = [row for row in election_reader]
f.close()

In the raw file, candidates and parties are in the same field, split by a dash.  Separate into two fields

In [105]:
candidate_row = rows[1]
candidate_fields = [field.split(' - ') for field in candidate_row]
candidates = [field[0] for field in candidate_fields]
parties = [field[1] if len(field) == 2 else field[0] for field in candidate_fields]


There are missing years in the data -- the year is only present in the first cell of a year.  Fill in the rest.  Also, convert each actual year to an integer.

In [106]:
years = rows[0]
last_year = years[0]
for i in range(1, len(years)):
    if (years[i] == ''):
        years[i] = last_year
    else:
        last_year = years[i]
years = ['Years'] + [int(year) for year in years[1:]]


A record for each state and year.  We are going to (1) create the state and year; (2) add the votes for each candidate to the record, putting total in total,
and then (3) create the percentage once everything has been read.  Note that we're going to trim records with zero total before creating the percentages.  As a side effect, when adding percentages we also add state and year to the record, because this is what we'll want in the row.

In [107]:
class StateAndYear:
    def __init__(self, state, year):
        self.year = int(year)
        self.state = state
        self.candidates = []
        self.total = 0
        
    def add_candidate(self, candidate, party, votes):
        if (candidate == 'Total'):
            self.total = votes
        else:
            self.candidates.append([candidate, party, votes])
    
    def add_percentages(self):
        self.candidates = [[self.state, self.year, cand[0], cand[1], cand[2], round(100 * cand[2]/self.total, 1), round(cand[2]/self.total, 3)] for cand in self.candidates]

Create the state and year records, sticking them in a dictionary indexed by state and year.  Note that votes are converted to int before being added to the record.

In [108]:
state_and_year_dictionary = {}
year_set  = set(years[1:])
for row in rows[2:]:
    state = row[0]
    for year in year_set:
        state_and_year_dictionary[(state, year)] = StateAndYear(state, year)
    for index in range(1, len(row)):
        try:
            votes = int(row[index])
            year = years[index]
            candidate = candidates[index]
            party = parties[index]
            state_and_year_dictionary[(state, year)].add_candidate(candidate, party, votes)
        except ValueError:
            pass

Trim the records with 0 total, add the percentages, collect and sort the records, and add a header

In [109]:
record_list = [record for record in state_and_year_dictionary.values() if record.total > 0]
for record in record_list:
    record.add_percentages()
    
data = []
for record in record_list:
    data = data + record.candidates

party_aliases = {"Republican": {"Republican", 'National Republican','National Union (Republican)', 'Whig'}, "Democratic": {'Democratic', 'Liberal Republican/Democratic', '(Northern) Democratic'}}

def unalias_party(party):
    return "Republican" if party in party_aliases["Republican"] else "Democratic" if party in party_aliases["Democratic"] else party

def compare_parties(party1, party2):
    rp1 = unalias_party(party1)
    rp2 = unalias_party(party2)
    if (rp1 == 'Democratic'):
        return -1
    if (rp2 == 'Democratic'):
        return 1
    if (rp1 == 'Republican'):
        return -1
    if (rp2 == 'Republican'):
        return 1
    return -1 if party1 < party2 else 1

def compare_states(state1, state2):
    if (state1 == 'Nationwide'): return -1
    if (state2 == 'Nationwide'): return 1
    return -1 if state1 < state2 else 1
    
    
import functools
def mycmp(record1, record2):
    if (record1[1] == record2[1]):
        if (record1[0] == record2[0]):
            return compare_parties(record1[3], record2[3])
        return compare_states(record1[0], record2[0])
    return -1 if record1[1] < record2[1] else 1

data.sort(key = functools.cmp_to_key(mycmp))

In [110]:

schema = [("State", "string"), ("Year", "number"), ("Candidate", "string"), ("Party", "string"), ("Votes", "number"), ("Percentage","number"), ("Pct", "number")]
# At some point try ("Year", "number", "Year", {f: "####"}) to get proper formatting
data_table = gviz_api.DataTable(schema)
data_table.LoadData(data)
google_table = data_table.ToJSon(columns_order=("State", "Year", "Candidate", "Party", "Votes", "Percentage", "Pct"))
send_data_to_dashboard("presidential_vote", google_table, client, room)


Prepare the pivot table.  Dig out all of the values for the parties from the original record list, then create a sorted list with the major parties 
as the first two, and the remainder alphabetic.


In [111]:
(len(test_client.log), test_client.log[-3:-1])

(3, [True, 'Client Connected'])

In [112]:
def parties(state_and_year):
    return set([record[3] for record in state_and_year.candidates])
all_parties = parties(record_list[0])
for record in record_list[1:]:
    all_parties = all_parties.union(parties(record))
party_list = list(all_parties)
party_list.sort(key = lambda p1:  -1 if p1 == 'Democratic' else 0 if p1 == 'Republican' else 1)

Construct the pivot table.   This takes us from a table which is (Year, State, Party, Percentage) to (Year, State, Democratic, Republican....) where the values in each party column are the percentages for that party for that state and year.  

In [113]:
def make_pivot_row(record):
    result = [0 for party in party_list]
    for candidate in record.candidates:
        index = party_list.index(candidate[3])
        result[index] = candidate[5]
    first_candidate = record.candidates[0]
    result = [first_candidate[0], first_candidate[1]] + result
    return result
pivot_table_rows = [make_pivot_row(record) for record in record_list]

Construct the pivot table schema, create the data table, and send it to the dashboard

In [114]:
pivot_schema = [("State", "string"), ("Year", "number")] + [(party, "number") for party in party_list]
pivot_table = gviz_api.DataTable(pivot_schema)
pivot_table.LoadData(pivot_table_rows)
google_pivot_table = pivot_table.ToJSon(columns_order=tuple(["Year", "State"] + party_list),
                           order_by=( "Year", "State"))
send_data_to_dashboard("presidential_vote_by_party", google_pivot_table, client, room)

Compute the margin table.  This will be a table of schema (Year, State, Margin) where Margin ranges from +5 to -5, integers.  Democratic margins are positive, Republican negative.  The margin is the actual margin divide by 2, capped at 5.  Where a third-party candidate won, the margin is 0.
Candidates have run under different banners, particularly in the 19th Century, so we normalize candidates to Republican, Democratic, or Other using
an aliases table, and then compute the margin in a straightforward way

In [115]:
aliases = {"Republican": {"Republican", 'National Republican','National Union (Republican)', 'Whig', 'Constitutional Union'}, "Democratic": {'Democratic', 'Liberal Republican/Democratic', '(Northern) Democratic','Southern Democratic'}}

def make_color_record(record):
    result = [0, 0, 0]
    for candidate in record.candidates:
        party = candidate[3]
        index = (0 if party in aliases["Democratic"] else 1 if party in aliases["Republican"] else 2)
        result[index] += candidate[5]
    off_scale = result[2] > result[1] and result[2] > result[0]
    absolute_margin = 0 if off_scale else result[0] - result[1]
    margin = min(5, max(round(absolute_margin/2), -5))
    reference = record.candidates[0]
    # reference[0] is the year, reference[1] is the state
    return [reference[0], reference[1], margin]

In [116]:
color_data = [make_color_record(record) for record in record_list]
# we are only interested in state data, so throw out "Nationwide"
color_data = [record for record in color_data if record[0] != 'Nationwide']

Set the schema for the margin table, create the table from the schema, load the rows (which are in color_data) and send to the dashboard.

In [117]:
color_schema = [("State", "string"), ("Year", "number"), ("Margin", "Number")]
color_table = gviz_api.DataTable(color_schema)
color_table.LoadData(color_data)
google_color_table = color_table.ToJSon(columns_order = ("Year", "State", "Margin"), order_by=("Year", "State"))
send_data_to_dashboard("presidential_vote_margins", google_color_table, client, room)

Create and send to the dashboard the electoral college table.  This involves primarily normalizing the party lists, and then writing out a table with the schema (Year, Democratic, Republican, ...), where each column contains the electoral votes for that party in that year.

In [118]:
f = open('electoral_college.csv')

In [119]:
import csv
ec_reader = csv.reader(f)
ec_records = [row for row in ec_reader]

In [120]:
def real_party(party):
    party_map = [('', 'Other'), ( 'Democratic', 'Democratic'), ( 'National Republican', 'Whig'), ( 'American (Know-Nothing)', 'Know-Nothing', ),
                 ( 'American Independent', 'American Independent'), ( 'Anti-Jackson', 'Anti-Jackson'), ( 'Anti-Masonic', 'Anti-Masonic'),
                 ( 'Constitutional Union', 'Constitutional Union'), ( 'Democrat', 'Democratic', ), ( 'Democratic', 'Democratic', ),
                 ( 'Democratic/Liberal Republican', 'Democratic/Liberal Republican'), ( 'Independent-Democratic', 'Democratic'), ( 'Libertarian', 'Libertarian'),
                 ( 'Nullifiers', 'Nullifiers', ), ( "People's (Populist)", "People's (Populist)"), ( 'Progressive', "Progressive"),
                 ( 'Progressive (Bull Moose)', "Progressive"), ( 'Republican', "Republican"), ( 'Southern Democratic', "Southern Democratic", ),
                 ( "States' Rights Democratic (Dixiecrat)", "Dixiecrat"), ( 'Whig', "Whig"), ( 'not a candidate', "Other")]
    match = [map for map in party_map if party == map[0]]
    if (len(match) == 0):
        return None
    return match[0][1]

The input schema is in the form (year, candidate, party, votes) and we want a schema in the form (year, party, votes) where party has been normalized using real_party.

In [121]:
output_records = [[int(record[0]), real_party(record[2]), int(record[3])] for record in ec_records[1:]]
output_records
ec_schema = [('Year', 'number'), ('Party', 'string'), ('Votes', 'number')]
ec_table = gviz_api.DataTable(ec_schema)

ec_table.LoadData(output_records)
google_ec_table = ec_table.ToJSon()
send_data_to_dashboard("electoral_college", google_ec_table, client, room)